In [ ]:
"""A generic DataModule for fine-tuning."""

import os
import shutil

import numpy as np
import pytorch_lightning as pl
from PIL import Image
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torchvision.utils import make_grid


def add_margin(pil_img, top, right, bottom, left):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height))
    result.paste(pil_img, (left, top))
    return result


def make_meshgrids(data_dir, grid_size=(4, 4), margin=16, out_name="meshgrid"):
    # n_img = grid_size[0] * grid_size[1]
    splits = os.listdir(f"{data_dir}")
    for split in splits:
        classes = os.listdir(f"{data_dir}/{split}")
        for cls in classes:
            meshes = os.listdir(f"{data_dir}/{split}/{cls}")
            for mesh in meshes:
                img_dir = f"{data_dir}/{split}/{cls}/{mesh}"
                img_names = os.listdir(img_dir)
                img_names.sort()
                if grid_size == (4, 4):
                    imgs = [read_image(f"{img_dir}/{img_fn}") for img_fn in img_names[1::2]]
                grid = make_grid(imgs, nrow=grid_size[1], padding=0)
                img_grid = transforms.ToPILImage()(grid)
                img_grid = img_grid.resize((224, 224))
                img_grid = add_margin(img_grid, margin, margin, margin, margin)

                out_dir = f"{out_name}/{split}/{cls}"
                os.makedirs(f"{out_dir}", exist_ok=True)
                img_grid.save(f"{out_dir}/{split}_{mesh}_{grid_size[0]}x{grid_size[1]}.png")

In [ ]:
make_meshgrids(
    "/home/dennis/Desktop/work/modelnet40_meshes", out_name="/home/dennis/Desktop/work/modelnet40meshgrid-4x4"
)

## Restructure images split/class/images

* add leading zeros to renders
* save images in split/class directory

In [ ]:
import os
import shutil

IN_DIR = "/home/dennis/Desktop/modelnet40_renders_32"
OUT = "/home/dennis/Desktop/work/modelnet40_meshes"
for path, dirnames, filenames in os.walk(f"{IN_DIR}"):
    for fname in filenames:
        fname_split = fname.split("_")
        split = fname_split[0]
        label = fname_split[1]
        mesh = f"{fname_split[1]}_{fname_split[2]}"
        # We just hack this in as 'night_stand' is the only two-word-class using _
        if label in ["night", "flower", "glass", "range", "tv"]:
            label = f"{fname_split[1]}_{fname_split[2]}"
            mesh = f"{fname_split[1]}_{fname_split[2]}_{fname_split[3]}"
        n_img = fname_split[-1].split(".")[0].zfill(4)
        ext = fname_split[-1].split(".")[1]
        out_dir = f"{OUT}/{split}/{label}/{mesh}"
        out_path = f"{out_dir}/{split}_{mesh}_{n_img}.{ext}"
        os.makedirs(out_dir, exist_ok=True)
        shutil.copy(f"{path}/{fname}", out_path)